In [ ]:
%load_ext autoreload
%autoreload 2

from nbdev import *
from fastcore.test import *
from rory import * # printnl, printdict
%nbdev_default_export bboxes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Cells will be exported to rory.bboxes,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
from fastai2.vision.all import *

# Bounding boxes from scratch
> Working my way up from single object classification to multi object segmentation with fastai2.

In [ ]:
datafolder = Path('/home/rory/repos/coco2020/data')
zipfiles = zipsfolder.ls(); zipfiles

(#9) [Path('/home/rory/repos/coco2020/data/image_info_test2017.zip'),Path('/home/rory/repos/coco2020/data/train2017.zip'),Path('/home/rory/repos/coco2020/data/test2017'),Path('/home/rory/repos/coco2020/data/val2017'),Path('/home/rory/repos/coco2020/data/train2017'),Path('/home/rory/repos/coco2020/data/annotations_trainval2017.zip'),Path('/home/rory/repos/coco2020/data/annotations'),Path('/home/rory/repos/coco2020/data/test2017.zip'),Path('/home/rory/repos/coco2020/data/val2017.zip')]

Import deps for unzip().

In [ ]:
from zipfile import *
from pathlib import *

In [ ]:
def unzip(*posixpaths,dest):
    """Unzips files located at posixpaths to dest. Deps: zipfile, pathlib."""
    for path in posixpaths:
        if path.suffix == '.zip':
            print(f"Unzipping: {path} ---> {dest}.")
            with ZipFile(path, 'r') as archive:
                archive.extractall(dest)
    print("Finished. Files in dest:")
    printnl(*dest.ls(), 1)

Uncomment the cell below if this is the first time running.

In [ ]:
# unzip(*zipfiles, dest=datafolder)

Unzipping: /home/rory/repos/coco2020/data/image_info_test2017.zip ---> /home/rory/repos/coco2020/data.
Unzipping: /home/rory/repos/coco2020/data/train2017.zip ---> /home/rory/repos/coco2020/data.
Unzipping: /home/rory/repos/coco2020/data/annotations_trainval2017.zip ---> /home/rory/repos/coco2020/data.
Unzipping: /home/rory/repos/coco2020/data/test2017.zip ---> /home/rory/repos/coco2020/data.
Unzipping: /home/rory/repos/coco2020/data/val2017.zip ---> /home/rory/repos/coco2020/data.
unzip finished. Files in dest:
/home/rory/repos/coco2020/data/image_info_test2017.zip
/home/rory/repos/coco2020/data/train2017.zip
/home/rory/repos/coco2020/data/test2017
/home/rory/repos/coco2020/data/val2017
/home/rory/repos/coco2020/data/train2017
/home/rory/repos/coco2020/data/annotations_trainval2017.zip
/home/rory/repos/coco2020/data/annotations
/home/rory/repos/coco2020/data/test2017.zip
/home/rory/repos/coco2020/data/val2017.zip


Fastai's helper function for getting COCO formatted labels out of their weird json structure.

In [ ]:
imgs_t, lbl_bbox_t = get_annotations(datafolder/'annotations/instances_train2017.json')
imgs_v, lbl_bbox_v = get_annotations(datafolder/'annotations/instances_val2017.json')

In [ ]:
imgs_t = list(map(lambda x: Path(datafolder/'train2017'/x),imgs_t))
imgs_v = list(map(lambda x: Path(datafolder/'train2017'/x),imgs_v))

I'll use pandas to create the src dataset. I want the fields (file, labels, bboxes, is_val).

In [ ]:
import pandas as pd

I'll create a list for each field then zip 'em up.

In [ ]:
imgs = imgs_t + imgs_v
bboxes = [i[0] for i in lbl_bbox_t] + [i[0] for i in lbl_bbox_v]
labels = [i[1] for i in lbl_bbox_t] + [i[1] for i in lbl_bbox_v]
is_val = [False]*len(imgs_t) + [True]*len(imgs_v)

In [ ]:
test_eqs(1,1)

NameError: name 'test_eqs' is not defined

In [ ]:
test_eqs(len(is_val), len(imgs_t)+len(imgs_v))

NameError: name 'test_eqs' is not defined

In [ ]:
[i[1] for i in lbl_bbox_t]

117266

In [ ]:
lbl_bbox_t[0]

([[359.17, 146.17, 471.62, 359.74],
  [339.88, 22.16, 493.76, 322.89000000000004],
  [471.64, 172.82, 507.56, 220.92],
  [486.01, 183.31, 516.64, 218.29]],
 ['motorcycle', 'person', 'person', 'bicycle'])